In [1]:
!pip install -r requirements.txt

In [2]:
from IPython.display import display, HTML, IFrame
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from functools import reduce
import pandas as pd
import requests
import time
import json

In [3]:
def intersection(lst1, lst2): 
    return set(lst1).intersection(lst2)

def merge_objects(object1, object2):
    intersect_keys = intersection(object1.keys(), object2.keys())
    if len(intersect_keys) > 0:
        for key in intersect_keys:
            appriori_values = object1[key] if type(object1[key]) == list else [object1[key]]
            object2[key] = appriori_values + [object2[key]]
    object1.update(object2)
    return object1

def get_page_data(url):
    response = requests.get(url)

    base_url = "https://www.br104.com.br/wp-json/oembed/1.0/embed?url={url}"
    url = base_url.format(url=url)
    try:
        parsed = requests.get(url).json()

        soup = BeautifulSoup(response.text)
        article = soup.find("div", class_="artigo")
        metas = soup.find_all("meta")
        meta_1 = metas[1]
        del metas[1]

        metas = [meta.attrs for meta in metas if "property" in meta.attrs and ("article:" in meta["property"] or "og:" in meta["property"]) ]
        metas = list(map(lambda m: {m["property"].replace(':','_'): m["content"]} , metas))
        metas = reduce(lambda prev, cur: merge_objects(prev, cur), metas)
    #     metas['text'] = article.text.replace('\n','')
        metas['text'] = [p.text for p in article.find_all('p') if "+ Leia também:" not in p.text ]

        metas["author_name"] = parsed["author_name"].split("|")[0].strip() if "author_name" in parsed else "Undefined" 
        metas["author_url"] = parsed["author_url"] if "author_url" in parsed else "Undefined" 
        return metas
    except e:
        print(base_url)
        return e

In [4]:
# # retrieving links
links_df = pd.read_csv('links.csv')
link_values = links_df['link'].values
base_url = "https://www.br104.com.br/ultimas-noticias/" 
current_page = 0
pages= []
while True:
    current_page+=1
    url = base_url if current_page == 1 else "{}page/{}".format(base_url,current_page)
    print("Current Page: {}".format(url))
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    links = soup.find_all("a", class_="titlePageCategory")
    hrefs = list(map(lambda link: link['href'], links))
    if len(links) == 0:
        break
    
    pages += links
    
    if len(intersection(hrefs,link_values)) > 0:
        break
    time.sleep(1)

# creating links dataframe
link_dataset = list(map(lambda page: {"link":page['href'], "title":page.find('h1').text.strip(), "subtitle":page.find("p").text.strip()}, pages))
links_df = pd.concat([links_df, pd.DataFrame(link_dataset)])
links_df = links_df.drop_duplicates()
links_df.to_csv('links.csv',index=False)

Current Page: https://www.br104.com.br/ultimas-noticias/
Current Page: https://www.br104.com.br/ultimas-noticias/page/2
Current Page: https://www.br104.com.br/ultimas-noticias/page/3
Current Page: https://www.br104.com.br/ultimas-noticias/page/4
Current Page: https://www.br104.com.br/ultimas-noticias/page/5
Current Page: https://www.br104.com.br/ultimas-noticias/page/6


In [5]:
# getting pages
links_df = pd.read_csv("links.csv")
br104_df = pd.read_csv("br104.csv")
links = links_df['link'].values
links_to_scrap = [link for link in links if link not in br104_df['og_url'].values]

data = []
for url in tqdm(links_to_scrap):
    data.append(get_page_data(url))


In [6]:
import pandas as pd
df = pd.DataFrame(data)

br104_df = pd.concat([br104_df, df])
br104_df = br104_df.sort_values(by=["article_published_time"], ascending=False)
br104_df = br104_df.drop_duplicates(subset='og_url', keep="last")
br104_df.to_csv("br104.csv", index= False, header=True)